In [1]:
import spacy 
import numpy as np
import emb_spacy
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, TimeDistributed, Bidirectional


from emb_spacy import get_embedding
from input import count_examples_and_max_length, pad_sentences_from_file
import label

776469


2024-04-26 16:05:29.473195: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-26 16:05:29.474903: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-26 16:05:29.578345: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-26 16:05:29.957728: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-26 16:05:31.385801: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

### Lecture des données 

In [2]:
# Ouverture en lecture des données
with open("train_corpus", "r", encoding="utf-8") as file:
    data = file.readlines()

# Compter les exemples et trouver la taille maximale
num_examples, MAX_SEQ_SIZE = count_examples_and_max_length(data)

# Affichage des résultats
print("Nombre d'exemples :", num_examples) 
print("Taille maximale de la phrase :", MAX_SEQ_SIZE)

vec_word, sortie = pad_sentences_from_file("train_corpus", MAX_SEQ_SIZE)
print("vecteur : ", vec_word.shape)
# vec_word.shape : (3945, 41)
# sortie.shape : (3945, 41)

Nombre d'exemples : 3945
Taille maximale de la phrase : 41
vecteur :  (3945, 41)


### Création des vecteurs d'entrée et de sortie

In [3]:
# Création du vecteur d'entrée 
entree = np.zeros((num_examples, MAX_SEQ_SIZE, 300))
for i, sentence in enumerate(vec_word):
    for j, word in enumerate(sentence):
        if word:
            entree[i, j] = get_embedding(word)
        else:
            entree[i, j] = np.zeros(300)

print("entree shape : ", entree.shape)
print("sortie shape : ", sortie.shape)

entree shape :  (3945, 41, 300)
sortie shape :  (3945, 41)


In [ ]:
# Ajustement du vecteur de sortie
# TODO : faut mettre en vecteur numérique chacune des étiquettes
sortie_one_hot = np.array([[[label.get_vector_from_label(label_i) for label_i in labels] for labels in sentence_labels] for sentence_labels in sortie])

print(sortie_one_hot[0][0])

### Modèle LSTM

In [4]:
labels = label.extract_label("atis.train")
#print("LABELS :", labels)
nbLabels = len(labels)  # Nombre d'étiquettes potentielles
embedding_size = len(entree[0][0])

tailleDictionnaire = emb_spacy.get_size_dict()  # Taille du dictionnaire de mots
config = {
    'hidden_size': 128, # Taille de la couche cachée du RNN
    'dropout_ rate': 0.5,  # Taux de dropout
    'nb_labels': nbLabels
}  

# Définir l'entrée du modèle
# pas besoin de mettre le nb d'ex car il les fait passer un par un 
input_layer = Input(shape=(MAX_SEQ_SIZE, embedding_size), dtype='float32')

# Ajouter une couche LSTM bidirectionnelle
X = Bidirectional(LSTM(units=config['hidden_size'], return_sequences=True))(input_layer)
X = Dropout(config['dropout_ rate'])(X)
# X = LSTM(units = 128)(X)
# X = Dropout(config['dropout_ rate'])(X)

X = Dense(units=config['nb_labels'])(X)
X = Activation(activation='softmax')(X)
print('config ok')

# Création du model
model = Model(inputs=input_layer, outputs=X)
print("create ok")

# Compilation du modèle
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print("compil ok")
print(model.summary())

config ok
create ok
compil ok


2024-04-26 16:05:59.876308: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-26 16:05:59.876929: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 41, 300)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 41, 256)        │       439,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 41, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 41, 81)         │        20,817 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 41, 81)         │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 460,113 (1.76 MB)

 Trainable params: 460,113 (1.76 MB)

 Non-trainable params: 0 (0.00 B)

None


### Entraînement du modèle LSTM
- x : entree de la forme nb_example x MAX_SEQ_SIZE x embedding_size
- y : sortie de la forme nb_example x MAX_SEQ_SIZE
- batch_size : nb d'échantillon à utiliser à chaque itération lors de l'entrainement
    - un batch_size + gd = accélère l'entrainement mais besoin de + de mémoire GPU
    - plus petit = ralentit l'entrainement mais meilleure convergence du modèle
- epochs : nb d'itération sur l'ens des données d'entrainement 
- validation_split : spécifie le fraction des données à utiliser comme données de validation

In [ ]:
model.fit(x=entree, y=sortie, batch_size=128, epochs=10, validation_split=0.2)
print('training ok')